# Scraping olx cars data by using requests and beautiful soup!

In [1]:
# importing packages/modules/libraries

import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
# function to get the data
def get_the_data(my_dict, new_soup):
    
    # getting the data from html content
    new_soup = new_soup.find_all('div', attrs={'class':'b44ca0b3'})
    
    # getting the span elements data    
    sublists = []
    for d in new_soup:
        sublists.append(d.find_all('span'))
    
    # iterating apn elements lists    
    for sublist in sublists:
        
        # adding key
        key = sublist[0].text.strip()
        
        # adding value
        value = sublist[1].text.strip()
        
        # initialize the list for the key if not present
        if key not in my_dict:
            my_dict[key] = []
        
        # append the value to the list associated with the key
        if value is None or value == "":
            my_dict[key].append("")
        else:
            my_dict[key].append(value)
        
    return my_dict

# get title function

def get_title(soup):
    try:
        title = soup.find('h1', attrs={'class':'a38b8112'}).string.strip()
    except AttributeError:
        title = ""
        
    return title

In [3]:
if __name__ == "__main__":
    
    # url of the webpage to be scraped
    URL = "https://www.olx.com.pk/cars_c84"
    
    # defining headers to be sent to the webpage
    HEADERS = ({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36', 'Accept-Language':'en-US, en;q=0.5'})

    # sending http get request to the webpage
    webpage = requests.get(URL, headers=HEADERS)
    
    # get the content from the webpage, parse into html and give a structure to it using beautifulsoup
    soup = BeautifulSoup(webpage.content, 'html.parser')
    
    # getting all div tags containing links
    links = soup.find_all('div', attrs={'class':'_9bea76df'})
    
    # getting all anchor tags containing links
    final_links = []
    for link in links:
        final_links.append(link.find('a'))
    
    # getting all the anchor links in the links_list
    links_list = []
    for link in final_links:
        links_list.append(link.get('href'))  
    
    # creating en emtpy dictionary to hold the extracted data
    my_dict = {"Title":[]}
    
    # getting all the keys to be added in the dictionary
    for link in links_list:
        
        # generating new url
        URL = "https://www.olx.com.pk" + link
        
        # again sending get request to this new url
        new_webpage = requests.get(URL, headers=HEADERS)
        
        # get the content from the webpage, parse into html and give a structure to it using beautifulsoup
        new_soup = BeautifulSoup(new_webpage.content, 'html.parser')
        
        # calling the function
        get_the_data(my_dict, new_soup)
    
    # getting links one by one and extracting details from those links
    for link in links_list:
        
        # generating new url
        URL = "https://www.olx.com.pk" + link
        
        # again sending get request to this new url
        new_webpage = requests.get(URL, headers=HEADERS)
        
        # get the content from the webpage, parse into html and give a structure to it using beautifulsoup
        new_soup = BeautifulSoup(new_webpage.content, 'html.parser')
        
        # get qll the data values
        my_dict["Title"].append(get_title(new_soup))
    
    # deleting this
    del my_dict["KM's driven"]
    
    # Using pandas to create dataframe and creating csv file!
    olx_data = pd.DataFrame.from_dict(my_dict)
    olx_data.to_csv(r'C:\Users\Haseeb Cheema\Desktop\data\projects\pythonprojects\webscraping\bs4-requests-pandas\olxcarsdata\olxcarsdata.csv', index=False)
    